<a href="https://colab.research.google.com/github/Javier19-cmd/Lab5-SDS/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 5

Javier Valle 20159

In [5]:
#from Archivos.clasificador import clasificacion
from clasificador import clasificacion
import pandas as pd
import json
import random
import re
import numpy as np

### Parte 1

Leyendo y guardando el tráfico que es DNS nada más

In [ ]:
with open('./large_eve.json', 'r') as archivo_json, \
        open('./dns_trafico.json', 'w') as archivo_dns:

    for linea in archivo_json:
        datos = json.loads(linea)
        if datos['event_type'] == 'dns':
            json.dump(datos, archivo_dns)
            archivo_dns.write('\n')

archivo_json.close()
archivo_dns.close()

Enseñando la cantidad de registros

In [3]:
with open('./dns_trafico.json', 'r') as archivo_dns:
    contador_registros = 0

    for linea in archivo_dns:
        contador_registros += 1

print("Cantidad de registros guardados:", contador_registros)

Cantidad de registros guardados: 15749


Enseñando la información de 2 registros cualesquiera

In [6]:
lineas = []

with open('./dns_trafico.json', 'r') as archivo_dns:
    for linea in archivo_dns:
        lineas.append(linea)

# Escogiendo dos índices aleatorios dentro del rango de líneas en el archivo
indice1 = random.randint(0, len(lineas)-1)
indice2 = random.randint(0, len(lineas)-1)

# Obteniendo los datos de los registros correspondientes a los índices escogidos
registro1 = json.loads(lineas[indice1])
registro2 = json.loads(lineas[indice2])


print("Registro 1:")
print(json.dumps(registro1, indent=4))
print("\nRegistro 2:")
print(json.dumps(registro2, indent=4))


Registro 1:
{
    "timestamp": "2017-07-22T19:29:18.711840-0500",
    "flow_id": 1103390249507751,
    "pcap_cnt": 3354173,
    "event_type": "dns",
    "vlan": 150,
    "src_ip": "192.168.207.4",
    "src_port": 53,
    "dest_ip": "192.168.205.188",
    "dest_port": 52340,
    "proto": "UDP",
    "dns": {
        "type": "answer",
        "id": 28302,
        "rcode": "NXDOMAIN",
        "rrname": "mirrors.tummy.com"
    }
}

Registro 2:
{
    "timestamp": "2017-07-22T17:36:28.422847-0500",
    "flow_id": 308404244214469,
    "pcap_cnt": 70786,
    "event_type": "dns",
    "vlan": 110,
    "src_ip": "192.168.207.4",
    "src_port": 53,
    "dest_ip": "192.168.201.68",
    "dest_port": 59948,
    "proto": "UDP",
    "dns": {
        "type": "answer",
        "id": 6926,
        "rcode": "NXDOMAIN",
        "rrname": "secure.informaction.com.localdomain"
    }
}


Normalizando la data

In [7]:
with open('./dns_trafico.json', 'r') as archivo_dns:
    datos = [json.loads(linea) for linea in archivo_dns]

# Normalizando los datos
df = pd.json_normalize(datos)

print("Shape del DataFrame:", df.shape)

Shape del DataFrame: (15749, 18)


Buscando dominios DGA

In [8]:
# Filtrando los registros DNS tipo A
registros_tipo_A = df[df['dns.rrtype'] == 'A']

print("Shape del DataFrame filtrado:", len(registros_tipo_A))

Shape del DataFrame filtrado: 2849


Filtrando los dominios únicos

In [9]:
# Conjunto para almacenar dominios únicos
unique_domains = set()

with open('./dns_trafico.json', 'r') as archivo_dns:
    for linea in archivo_dns:
        datos = json.loads(linea)
        # Extrayendo el dominio del campo 'rrname' y agregarlo al conjunto
        domain = datos['dns']['rrname']
        unique_domains.add(domain)

num_unique_domains = len(unique_domains)

print("Cantidad de registros únicos:", num_unique_domains)

Cantidad de registros únicos: 334


Escribiendo una función que obtengo el TLD para un dominio

In [10]:
def obtener_TLD(dominio):
    # Expresión regular para encontrar el TLD
    patron = r'\.(.*?)\.(?:com|org|net|gov|edu|int|mil|arpa|biz|info|name|pro|aero|coop|museum|jobs|travel|cat|tel|onion|git|home|local|example|test|invalid|localhost|localdomain)$'
    match = re.search(patron, dominio)
    if match:
        return match.group(1)
    else:
        return dominio

# Ejemplo de uso
dominio1 = "api.wunderground.com"
dominio2 = "safebrowsing.clients.google.com.home"

print("TLD de", dominio1, ":", obtener_TLD(dominio1))
print("TLD de", dominio2, ":", obtener_TLD(dominio2))

TLD de api.wunderground.com : wunderground
TLD de safebrowsing.clients.google.com.home : clients.google.com


Prompt utilizado: "hola, me podrías escribir una función que me obtenga el TLD para un dominio"

Creando una columna que se llame domain_tld y eliminando las demás columnas

In [11]:
if 'domain_tld' not in df.columns:
    df['domain_tld'] = df['dns.rrname'].apply(obtener_TLD)

    df = df[['domain_tld']]

print(df)

                        domain_tld
0                     wunderground
1                          dropbox
2      office.aol.com.ad.aol.aoltw
3      office.aol.com.ad.aol.aoltw
4                           <root>
...                            ...
15744               start3.mozilla
15745                       <root>
15746        whitecell.localdomain
15747        whitecell.localdomain
15748                       <root>

[15749 rows x 1 columns]


### Parte 2

Usando el clasificador proporcionado y creando un nuevo dataframe

In [18]:
df

result = clasificacion(df)

result

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,domain_tld,isDGA
0,wunderground,1
1,dropbox,0
2,office.aol.com.ad.aol.aoltw,1
3,office.aol.com.ad.aol.aoltw,1
4,<root>,0
...,...,...
15744,start3.mozilla,1
15745,<root>,0
15746,whitecell.localdomain,0
15747,whitecell.localdomain,0


Filtrando aquellos que son DGA y enseñándolos.

In [21]:
# Filtrando aquellos considerados como DGA (valor 1)
result = result[result['isDGA'] == 1]

print("Registros considerados como DGA:")
print(result)

# Eliminando duplicados
result_unique = result.drop_duplicates()


print("\nCantidad de registros únicos después de eliminar duplicados:", len(result_unique))

Registros considerados como DGA:
                              domain_tld  isDGA
0                           wunderground      1
2            office.aol.com.ad.aol.aoltw      1
3            office.aol.com.ad.aol.aoltw      1
5                           wunderground      1
21           office.aol.com.ad.aol.aoltw      1
...                                  ...    ...
15735  _dns-sd._udp.0.195.16.172.in-addr      1
15740                 26.168.192.in-addr      1
15741                 26.168.192.in-addr      1
15743                     start3.mozilla      1
15744                     start3.mozilla      1

[3525 rows x 2 columns]

Cantidad de registros únicos después de eliminar duplicados: 71


### Parte 3 - Dominio experto

In [ ]:
def verificar_TLD(tld, tld_list):
    if tld in tld_list:
        return 0
    else:
        return 1